In [210]:
import numpy as np
import pandas as pd
import datetime
import re
from bs4 import BeautifulSoup  #HTML 파싱
from urllib.parse import quote  #검색어 인코딩
import time  #슬립 사용
import requests  #HTML 가져오기

데이터 가져오기

contents log_dt - vod disp_rm

In [211]:
vod_movie = pd.read_csv('./LG_data/01. 전처리/vod_movie_전처리.csv', encoding='cp949')
contents_movie = pd.read_csv('./LG_data/01. 전처리/contents_movie_전처리.csv', encoding='cp949')


vod_movie.drop(columns= 'Unnamed: 0', axis= 1, inplace= True)
contents_movie.drop(columns= 'Unnamed: 0', axis= 1, inplace= True)

print(vod_movie.info()) #3852
print(contents_movie.info()) #17466


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3852 entries, 0 to 3851
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ID                  3852 non-null   int64 
 1   title               3852 non-null   object
 2   category            3852 non-null   object
 3   genre_category      3852 non-null   object
 4   viewing_time        3852 non-null   object
 5   SMRY                3852 non-null   object
 6   cast                3852 non-null   object
 7   running_time        3852 non-null   object
 8   viewing_start_time  3852 non-null   object
 9   viewing_start_date  3852 non-null   object
dtypes: int64(1), object(9)
memory usage: 301.1+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17466 entries, 0 to 17465
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0.1    17466 non-null  int64  
 1   ID         

In [212]:
print(contents_movie.head())
vod_movie.head()

   Unnamed: 0.1        ID  series_nm    title category genre_category  \
0             0  61779000        NaN      마들렌       영화             멜로   
1             1  67170000        NaN  1번가의 기적       영화            코미디   
2             2  66547000        NaN  1번가의 기적       영화            코미디   
3             3  66547000        NaN  1번가의 기적       영화            코미디   
4             4  64300000        NaN  1번가의 기적       영화            코미디   

                                                SMRY  \
0  소설가를 꿈꾸는 지석. 헤어 디자이너를 꿈꾸는 희진. 몇 번의 우연한 만남이 이어지...   
1  재개발의 막중한 임무를 띠고 에쿠스를 끌며 폼 나게 1번가에 나타난 날건달, 필제....   
2  재개발의 막중한 임무를 띠고 에쿠스를 끌며 폼 나게 1번가에 나타난 날건달, 필제....   
3  재개발의 막중한 임무를 띠고 에쿠스를 끌며 폼 나게 1번가에 나타난 날건달, 필제....   
4  재개발의 막중한 임무를 띠고 에쿠스를 끌며 폼 나게 1번가에 나타난 날건달, 필제....   

                            cast running_time               log_dt   type  \
0        조인성,신민아,박정아,강래연,김수로,최규환     01:56:00  2024-01-24 22:12:13  movie   
1  임창정,하지원,주현,정두홍,이훈,강예원,박창익,박유선     01:52:00  2024-01-05 09:08:08  mo

,ID,title,category,genre_category,viewing_time,SMRY,cast,running_time,viewing_start_time,viewing_start_date
0,62248000,(FREE)1번가의 기적(무료),영화,드라마,00:00:12,"재개발의 막중한 임무를 띠고 에쿠스를 끌며 폼 나게 1번가에 나타난 날건달, 필제....","임창정,하지원,주현,정두홍,이훈,강예원,박창익,박유선",01:52:00,16:58:47,2024-03-30
1,66547000,(FREE)1번가의 기적(무료),영화,드라마,01:52:00,"재개발의 막중한 임무를 띠고 에쿠스를 끌며 폼 나게 1번가에 나타난 날건달, 필제....","임창정,하지원,주현,정두홍,이훈,강예원,박창익,박유선",01:52:00,16:58:47,2024-03-30
2,67170000,(FREE)1번가의 기적(무료),영화,드라마,00:00:00,"재개발의 막중한 임무를 띠고 에쿠스를 끌며 폼 나게 1번가에 나타난 날건달, 필제....","임창정,하지원,주현,정두홍,이훈,강예원,박창익,박유선",01:52:00,16:58:47,2024-03-30
3,66056000,(FREE)1번가의 기적(무료),영화,드라마,00:00:11,"재개발의 막중한 임무를 띠고 에쿠스를 끌며 폼 나게 1번가에 나타난 날건달, 필제....","임창정,하지원,주현,정두홍,이훈,강예원,박창익,박유선",01:52:00,16:58:47,2024-03-30
4,62248000,(FREE)1번가의 기적(무료),영화,드라마,00:00:21,"재개발의 막중한 임무를 띠고 에쿠스를 끌며 폼 나게 1번가에 나타난 날건달, 필제....","임창정,하지원,주현,정두홍,이훈,강예원,박창익,박유선",01:52:00,16:58:47,2024-03-30


시계열 데이터 변환

In [200]:
# vod_movie['viewing_time'] = pd.to_datetime(vod_movie['viewing_time']).dt.time
# vod_data['viewing_time'] = viewing_data

In [239]:
# 동일한 ID, title, start_date의 viewing_start_time - log_time
# DataFrame.compare(other, align_axis=1, keep_shape=False, keep_equal=False)
# viewing_start_time - log_dt 간의 시간이 길 수 있음 : 상세페이지에 들어가놓고 나중에 본 경우

log_vod_movie = vod_movie[['ID', 'title', 'viewing_start_date', 'viewing_start_time', 'viewing_time']]
log_contents_movie = contents_movie[['ID', 'title','log_date', 'log_time']]

#시청 페이지의 viewing_start_date와 상세 페이지의 log_date 가 동일한 일자여야 함
log_vod_movie.columns = ['ID', 'title', 'start_date', 'viewing_start_time', 'viewing_time']
log_contents_movie.columns = ['ID', 'title', 'start_date', 'log_time']

print(log_vod_movie.info()) #3852
print(log_contents_movie.info()) #17466


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3852 entries, 0 to 3851
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ID                  3852 non-null   int64 
 1   title               3852 non-null   object
 2   start_date          3852 non-null   object
 3   viewing_start_time  3852 non-null   object
 4   viewing_time        3852 non-null   object
dtypes: int64(1), object(4)
memory usage: 150.6+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17466 entries, 0 to 17465
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ID          17466 non-null  int64 
 1   title       17466 non-null  object
 2   start_date  17466 non-null  object
 3   log_time    17466 non-null  object
dtypes: int64(1), object(3)
memory usage: 545.9+ KB
None


상세페이지 log가 없는 경우 / viewing_start_time이 없는 경우 drop

In [259]:
#log_time이 없는 경우 상세페이지 없이 바로 vod 시청한 경우로 결측치 행 drop
log_contents_movie = log_contents_movie.drop(log_contents_movie[log_contents_movie['log_time'].isna()].index, axis=0)
# print(log_contents_movie.info())
# print(log_contents_movie.head())

#viewing_start_time이 없는 경우 상세페이지에서 전환이 안된 경우로 결측치 행 drop
log_vod_movie = log_vod_movie.drop(log_vod_movie[log_vod_movie['viewing_start_time'].isna()].index, axis=0)

log_vod_movie.info() 
print(log_vod_movie.head())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3852 entries, 0 to 3851
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ID                  3852 non-null   int64 
 1   title               3852 non-null   object
 2   start_date          3852 non-null   object
 3   viewing_start_time  3852 non-null   object
 4   viewing_time        3852 non-null   object
dtypes: int64(1), object(4)
memory usage: 150.6+ KB
         ID              title  start_date viewing_start_time viewing_time
0  62248000  (FREE)1번가의 기적(무료)  2024-03-30           16:58:47     00:00:12
1  66547000  (FREE)1번가의 기적(무료)  2024-03-30           16:58:47     01:52:00
2  67170000  (FREE)1번가의 기적(무료)  2024-03-30           16:58:47     00:00:00
3  66056000  (FREE)1번가의 기적(무료)  2024-03-30           16:58:47     00:00:11
4  62248000  (FREE)1번가의 기적(무료)  2024-03-30           16:58:47     00:00:21


vod-contents 데이터 결합

In [254]:
# 데이터 결합
# 데이터 길이 확인
log_total_movie = pd.concat([log_vod_movie, log_contents_movie], ignore_index = True, join = 'outer')
print(log_total_movie.info())
print(log_total_movie.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21318 entries, 0 to 21317
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ID                  21318 non-null  int64 
 1   title               21318 non-null  object
 2   start_date          21318 non-null  object
 3   viewing_start_time  3852 non-null   object
 4   viewing_time        3852 non-null   object
 5   log_time            17466 non-null  object
dtypes: int64(1), object(5)
memory usage: 999.4+ KB
None
         ID              title  start_date viewing_start_time viewing_time  \
0  62248000  (FREE)1번가의 기적(무료)  2024-03-30           16:58:47     00:00:12   
1  66547000  (FREE)1번가의 기적(무료)  2024-03-30           16:58:47     01:52:00   
2  67170000  (FREE)1번가의 기적(무료)  2024-03-30           16:58:47     00:00:00   
3  66056000  (FREE)1번가의 기적(무료)  2024-03-30           16:58:47     00:00:11   
4  62248000  (FREE)1번가의 기적(무료)  2024-03-30           16:58:47

In [253]:
log_total_movie.loc[(log_total_movie['log_time'].isna()), :]

,ID,title,start_date,viewing_start_time,viewing_time,log_time
0,62248000,(FREE)1번가의 기적(무료),2024-03-30,16:58:47,00:00:12,NaN
1,66547000,(FREE)1번가의 기적(무료),2024-03-30,16:58:47,01:52:00,NaN
2,67170000,(FREE)1번가의 기적(무료),2024-03-30,16:58:47,00:00:00,NaN
3,66056000,(FREE)1번가의 기적(무료),2024-03-30,16:58:47,00:00:11,NaN
4,62248000,(FREE)1번가의 기적(무료),2024-03-30,16:58:47,00:00:21,NaN
...,...,...,...,...,...,...
3847,60784000,환희,2024-03-30,16:58:47,00:02:16,NaN
3848,59899000,(SD)경마장가는길,2024-03-30,16:58:47,00:07:36,NaN
3849,66556000,괴물(2023)(예고),2024-03-30,16:58:47,00:00:56,NaN
3850,66556000,괴물(2023)(예고),2024-03-30,16:58:47,00:00:05,NaN


In [242]:
#자기 자신 반환
def original_value(x) :
  return x

log_total_movie.groupby(['ID', 'title', 'start_date']).agg(original_value)



viewing_start_time  \
ID       title      start_date                                                      
59880000 도그데이즈      2024-03-09                                                NaN   
59883000 마크맨        2024-02-15                                                NaN   
         서울의 봄      2024-02-15                                    [nan, nan, nan]   
         위험한상견례     2024-03-30                                           16:58:47   
         표적         2024-03-30                                           16:58:47   
...                                                                           ...   
67356000 청춘선거       2024-02-26                                                NaN   
67374000 드론 컨트롤러    2024-03-21                                                NaN   
         모닝 애프터     2024-03-21                                                NaN   
67378000 서울의 봄      2024-03-13  [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...   
67391000 사운드 오브 프리덤 2024-03-30                                         [nan, nan]   

                                                                     viewing_time  \
ID       title      start_date                                                      
59880000 도그데이즈      2024-03-09                                                NaN   
59883000 마크맨        2024-02-15                                                NaN   
         서울의 봄      2024-02-15                                    [nan, nan, nan]   
         위험한상견례     2024-03-30                                           00:00:00   
         표적         2024-03-30                                           00:21:50   
...                                                                           ...   
67356000 청춘선거       2024-02-26                                                NaN   
67374000 드론 컨트롤러    2024-03-21                                                NaN   
         모닝 애프터     2024-03-21                                                NaN   
67378000 서울의 봄      2024-03-13  [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...   
67391000 사운드 오브 프리덤 2024-03-30                                         [nan, nan]   

                                                                         log_time  
ID       title      start_date                                                     
59880000 도그데이즈      2024-03-09                                           09:22:41  
59883000 마크맨        2024-02-15                                           22:53:40  
         서울의 봄      2024-02-15                     [22:23:19, 22:23:13, 22:23:57]  
         위험한상견례     2024-03-30                                                NaN  
         표적         2024-03-30                                                NaN  
...                                                                           ...  
67356000 청춘선거       2024-02-26                                           10:04:08  
67374000 드론 컨트롤러    2024-03-21                                           07:28:24  
         모닝 애프터     2024-03-21                                           07:26:47  
67378000 서울의 봄      2024-03-13  [10:28:28, 10:30:31, 10:28:23, 10:28:23, 10:33...  
67391000 사운드 오브 프리덤 2024-03-30                               [16:07:13, 16:07:21]  

[12363 rows x 3 columns]

재석님

In [ ]:
for i in log_total_movie.groupby(['ID', 'title','start_date']).agg(original_value)['log_time']:
    print(type(i))
    # if type(i) == np.ndarray:
    #   print(list(i))
    # else:
    #   print(i)

In [ ]:
log_total_movie = log_total_movie.groupby(['ID', 'title', 'start_date']).agg(original_value)



In [136]:
def islist(list_series) :
  if isinstance(list_series, list) :
    return True
  else :
    return False


is_list_idx = log_existence_movie[((log_existence_movie['viewing_start_time'].apply(islist)) 
& (log_existence_movie['viewing_time'].apply(islist)) 
& (log_existence_movie['log_time'].apply(islist)))].index


SyntaxError: invalid syntax (4140209605.py, line 1)

In [144]:
print(log_existence_movie['viewing_start_time'].notnull())

ID        title       start_date
59880000  도그데이즈       2024-03-09    False
59883000  마크맨         2024-02-15    False
          서울의 봄       2024-02-15     True
59884000  노량: 죽음의바다   2024-02-11    False
59886000  서울의 봄       2024-02-17     True
                                    ...  
67356000  청춘선거        2024-02-26    False
67374000  드론 컨트롤러     2024-03-21    False
          모닝 애프터      2024-03-21    False
67378000  서울의 봄       2024-03-13     True
67391000  사운드 오브 프리덤  2024-03-30     True
Name: viewing_start_time, Length: 10799, dtype: bool


FREE VOD

예고편 - 영화-기타(성인물 포함) / 시리즈물 - 0분짜리에 일반 vod도 포함